In [2]:
import numpy as np
import h5py
import torch
from torch import nn
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [3]:
fn = "image1-competition.hdf5"

with h5py.File(fn, "r") as f:
    data = np.array(f["data"])
    wns = np.array(f["wns"])
    tissue_mask = np.array(f["tissue_mask"])
    classes = np.array(f["classes"])

In [4]:
X_test = data[475:700, 750:1000, :]
y_test = classes[475:700, 750:1000]

train_data = np.delete(data, slice(475, 700), axis=0)
X_train = np.delete(train_data, slice(750, 1000), axis=1)

train_data_y = np.delete(classes, slice(475, 700), axis=0)
y_train = np.delete(train_data_y, slice(750, 1000), axis=1)


In [5]:
annotated_sel_test = y_test != -1

X_test = X_test[annotated_sel_test, :]
y_test = y_test[annotated_sel_test]



annotated_sel_train = y_train != -1
X_train = X_train[annotated_sel_train]
y_train = y_train[annotated_sel_train]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [6]:
X_train.shape

torch.Size([25769, 187])

In [28]:
class NeuralNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(187, 15),
            nn.ReLU(),
            nn.Linear(15, 10),
            nn.ReLU(),
            nn.Linear(10, 6)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
def train(X, y, model, learning_rate, num_epochs, loss_fn, X_test, y_test, validation=False):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.001)
    model.train()
    torch.manual_seed(42)

    for epoch in range(num_epochs):
        
        # Forward pass
        preds = model(X)

        # Compute loss
        loss = loss_fn(preds, y)
        
        # Backward pass
        loss.backward()
        
        # Print loss after each epoch
        if (epoch + 1) % 10 == 0:  # Print every 10 epochs (adjust as needed)
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


        # Update model parameters
        optimizer.step()
        
        # Zero gradients for the next step
        optimizer.zero_grad()

        if validation:      
            # Validation step
            model.eval()  # Switch to evaluation mode (disables dropout, batch norm, etc.)
            with torch.no_grad():  # Disable gradient calculation for validation to save memory and computation
                val_preds = model(X_test)
                val_loss = loss_fn(val_preds, y_test)
            
            # Print validation loss after each epoch
            if (epoch + 1) % 10 == 0:  # Print every 10 epochs (adjust as needed)
                print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss.item():.4f}")
            
        model.train() 

    print("Training complete.")

def test(X_test, y_test, model, loss_fn):
    model.eval()  # Set model to evaluation mode (disables dropout, batch norm, etc.)
    
    # Forward pass
    with torch.no_grad():  # No gradient tracking needed for inference
        preds = model(X_test)
    # Convert predictions to class labels (for classification)
    _, predicted = torch.max(preds, 1)
    
    # Calculate accuracy
    correct = (predicted == y_test).sum().item()
    total = y_test.size(0)
    accuracy = 100 * correct / total
    
    # Calculate cross-entropy loss
    print(y_test.shape)
    loss = loss_fn(preds, y_test)
    
    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Test Loss (Cross-Entropy): {loss:.4f}")

In [ ]:
model = NeuralNetwork()
train(X_train, y_train, model, 0.001, 1200, nn.CrossEntropyLoss(), X_test, y_test, True)

Epoch [10/1200], Loss: 1.9000
Epoch [10/1200], Validation Loss: 1.8362
Epoch [20/1200], Loss: 1.8043
Epoch [20/1200], Validation Loss: 1.8121
Epoch [30/1200], Loss: 1.7092
Epoch [30/1200], Validation Loss: 1.7827
Epoch [40/1200], Loss: 1.6270
Epoch [40/1200], Validation Loss: 1.7478
Epoch [50/1200], Loss: 1.5571
Epoch [50/1200], Validation Loss: 1.7092
Epoch [60/1200], Loss: 1.4807
Epoch [60/1200], Validation Loss: 1.6630
Epoch [70/1200], Loss: 1.3872
Epoch [70/1200], Validation Loss: 1.6146
Epoch [80/1200], Loss: 1.2790
Epoch [80/1200], Validation Loss: 1.5570
Epoch [90/1200], Loss: 1.1661
Epoch [90/1200], Validation Loss: 1.5052
Epoch [100/1200], Loss: 1.0640
Epoch [100/1200], Validation Loss: 1.4795
Epoch [110/1200], Loss: 0.9788
Epoch [110/1200], Validation Loss: 1.5093
Epoch [120/1200], Loss: 0.9131
Epoch [120/1200], Validation Loss: 1.5764
Epoch [130/1200], Loss: 0.8596
Epoch [130/1200], Validation Loss: 1.6313
Epoch [140/1200], Loss: 0.8126
Epoch [140/1200], Validation Loss: 1.6

In [27]:
test(X_test, y_test, model, nn.CrossEntropyLoss())

torch.Size([5192])
Test Accuracy: 72.23%
Test Loss (Cross-Entropy): 0.8767


In [ ]:
annotated_sel = classes != -1
X_real = data[annotated_sel]
y_real = classes[annotated_sel]

scaler = StandardScaler()
X_real = scaler.fit_transform(X_real)

data_predict = data[265:465,360:660]

X_real = torch.tensor(X_real, dtype=torch.float32)
y_real = torch.tensor(y_real, dtype=torch.long)
train(X_real, y_real, model, 0.001, 1200, nn.CrossEntropyLoss(), [], [])

Epoch [10/100], Loss: 0.3279
Epoch [20/100], Loss: 0.3019
Epoch [30/100], Loss: 0.2860
Epoch [40/100], Loss: 0.2709
Epoch [50/100], Loss: 0.2588
Epoch [60/100], Loss: 0.2477
Epoch [70/100], Loss: 0.2384
Epoch [80/100], Loss: 0.2307
Epoch [90/100], Loss: 0.2243
Epoch [100/100], Loss: 0.2215
Training complete.


In [11]:
lin_data_predict = data_predict.reshape(-1, data_predict.shape[-1])
lin_data_predict = scaler.transform(lin_data_predict)

lin_data_predict = torch.tensor(lin_data_predict, dtype=torch.float32)
pred = model(lin_data_predict)
pred = torch.softmax(pred, dim=1)
pred = pred.reshape((200, 300, -1)).detach().numpy()
print(pred.shape)

with open("first.npy", "wb") as f:
    np.save(f, pred.astype(np.float32))

(200, 300, 6)
